In [5]:
import os, time
import numpy as np
import pandas as pd

from astropy.io import fits
from astropy.wcs import WCS
from astropy.stats import mad_std
from astropy.wcs.utils import pixel_to_skycoord

from photutils import CircularAperture, CircularAnnulus, SkyCircularAperture, aperture_photometry, DAOStarFinder, IRAFStarFinder

from astroquery.astrometry_net import AstrometryNet
from astroquery.exceptions import TimeoutError

from a345_utilities import print_header, mem_usage     # to use this in your own scripts copy the file a345_utilities.py to your work directory

# Directories

In [7]:
#Directories
data_dir = '/data/observatory/remote_telescope/data_derotated/'
 
#Directries for different size images
sub_dir_full = os.path.join(data_dir, 'full_size')
sub_dir_6000 = os.path.join(data_dir, 'cropped_6000x6000')
sub_dir_5000 = os.path.join(data_dir, 'cropped_5000x3000')
sub_dir_3000 = os.path.join(data_dir, 'cropped_3000x3000')
sub_dir_1800x1200 = os.path.join(data_dir, 'cropped_1800x1200')

#Setting to 3000x3000
sub_dir = sub_dir_3000

#Cluster directories:
M35_dir = 'M35'
M67_dir = 'M67'
M67v2_dir = 'M67v2'
NGC869_dir = 'NGC869'

#Setting Cluster to process
cluster_dir = NGC869_dir  #------------------------------------------!!!!!

#Directories to the calibration frames and the raw image files
calib_dir = os.path.join(data_dir, sub_dir, 'calibration/neg5c/master') 
img_dir = os.path.join(data_dir, sub_dir, 'images/Cluster Photometry', cluster_dir)

#Output directories to shared and local folders
shared_dir = os.path.join('/data/observatory/student_data/William_ODonnell', cluster_dir)
local_dir = cluster_dir

#Directories for the Calibration Frames
bias_fname = os.path.join(calib_dir,'bias_master.fits')
dark_120_fname = os.path.join(calib_dir,'dark_120s_master.fits')
dark_300_fname = os.path.join(calib_dir,'dark_300s_master.fits')
G_flat = os.path.join(calib_dir,'flat_G_master.fits') 
I_flat = os.path.join(calib_dir,'flat_I_master.fits')
R_flat = os.path.join(calib_dir,'flat_R_master.fits')

#Server to send source data to for plate solving - later
ast = AstrometryNet()
ast.API_URL = 'http://nova.astro.gla.ac.uk/api' # local server
ast.api_key = 'XXXXXXXX'
ast.URL = 'http://nova.astro.gla.ac.uk'
    
# Backup server on Ettus - use this as a back-up if there are issues with the primary server 
# ast.API_URL = 'http://ettus3.astro.gla.ac.uk:8000/api' # local server
# ast.URL = 'http://ettus3.astro.gla.ac.uk:8000'
# ast.api_key = 'XXXXXXXX'     

# Calibration

In [8]:
def calibrate(file):
    
    print("Calibrating...")
    
    #Reading in each file
    with fits.open(os.path.join(img_dir, file)) as hdu: 
        img_header = hdu[0].header
        img_data = hdu[0].data
        
    
    print("This image is in the ", img_header['FILTER'], "filter")
    
    #Opening the dark and bias frames
    with fits.open(bias_fname) as hdu:
        bias_header = hdu[0].header
        bias_data = hdu[0].data
    
    if img_header['EXPTIME'] == 120.0:
        dark_fname = dark_120_fname 
    elif img_header['EXPTIME'] == 300.0:
        dark_fname = dark_300_fname
    
    
    with fits.open(dark_fname) as hdu:
        dark_header = hdu[0].header
        dark_data = hdu[0].data

    with fits.open(G_flat) as hdu:
        G_flat_header = hdu[0].header
        G_flat_data = hdu[0].data
    with fits.open(I_flat) as hdu:
        I_flat_header = hdu[0].header
        I_flat_data = hdu[0].data
    with fits.open(R_flat) as hdu:
        R_flat_header = hdu[0].header
        R_flat_data = hdu[0].data


    if img_header['FILTER'] == "G'":
        average_flat_value = np.mean(G_flat_data - bias_data)
        img_data_corr = (img_data - dark_data) / (G_flat_data - bias_data) * average_flat_value
        
    elif img_header['FILTER'] == "I'":
        average_flat_value = np.mean(I_flat_data - bias_data)
        img_data_corr = (img_data - dark_data) / (I_flat_data - bias_data) * average_flat_value
        
    elif img_header['FILTER'] == "R'":
        average_flat_value = np.mean(R_flat_data - bias_data)
        img_data_corr = (img_data - dark_data) / (R_flat_data - bias_data) * average_flat_value
        
    calib_file = fits.PrimaryHDU(img_data_corr, header = img_header)
    
    return calib_file

# Plate Solve

In [10]:
def plateSolve(calib_file):
    
    print("Calibration Completed! Starting Plate Solve...")
    
    header = calib_file.header
    data = calib_file.data

#Defining different setting to be used for each filter  
    FILTER = header['FILTER']
    Fwhm = None
    Thresh = None
    if FILTER == "G'":
        Fwhm = 6
        thresh = 5
        radius = 1.5
        minsep = 1.0
    elif FILTER == "I'":
        Fwhm = 4
        thresh = 5
        radius = 1.5
        minsep = 1.0
    elif FILTER == "R'":
        Fwhm =  4
        thresh = 5
        radius = 1.5
        minsep = 1.0

#IRAF starfinder locates the sources in each image
    bkg_sigma = mad_std(data)
    IRAFfind_wcs = IRAFStarFinder(fwhm=Fwhm, threshold=thresh*bkg_sigma, sigma_radius=radius, minsep_fwhm=minsep, sharplo=0.5,
                         sharphi=2.0, roundlo=0.0, roundhi=0.5, sky=None, exclude_border= True, brightest= 250,peakmax=None)
    IRAFfind = IRAFStarFinder(fwhm=Fwhm, threshold=thresh*bkg_sigma, sigma_radius=radius, minsep_fwhm=minsep, sharplo=0.5,
                         sharphi=2.0, roundlo=0.0, roundhi=0.3, sky=None, exclude_border= True, brightest= None,peakmax=None)
    sources = IRAFfind_wcs(data) 
    sources.sort('flux')
    sources.reverse()  
    positions = np.transpose((sources['xcentroid'], sources['ycentroid']))
    
            
#Sending list of sources to server for each image
    image_width, image_height = data.shape
    wcs_header = None             
    t_start = time.time()
    try:
        print('Sending data to AstrometryNet server:')
        wcs_header = ast.solve_from_source_list(sources['xcentroid'], sources['ycentroid'], image_width, image_height, solve_timeout=2000)
                    
        if wcs_header:  
            print('\n -> Plate Solve Success! Solving took {:0.1f}s'.format(time.time()-t_start))
                       
        else:
            print('\n -> Solving failed after {:0.1f}s'.format(time.time()-t_start))
                  
    except TimeoutError:      
                print('\n -> ##FAIL: Timeout while solving, try a longer timeout, optmise number of sources (200-800 seems about right)')

#Updating header with WCS info
    header.update(wcs_header) 
    hdu = fits.PrimaryHDU()         
    hdu.header.update(header)      
    hdu.data = data    
        
    return hdu.header, hdu.data, IRAFfind

# Coord-List

In [5]:
RA = []; DEC = [];

for file in os.listdir(img_dir):
    if file.endswith(".fits"):
        print("----------------------------------------------------------------------")
        print("----------------------------------------------------------------------")
        print("Opening file: ", file)
        
#Calibrating and plate-solving each image
        header, data, IRAFfind = plateSolve(calibrate(file))
        wcs = WCS(header)
        name = "PlateSolved_Calib_" + (os.path.splitext(file)[0])
        PSfile = fits.PrimaryHDU(data, header = header)
        PSfile.writeto(os.path.join(shared_dir, name + '.fits'), overwrite=True)
        
# get some stats of the image
        d_mean = np.mean(data); d_med = np.median(data);    # mean and median of image
        d_std  = np.std(data);                              # standard deviation of intensity
        d_min = np.min(data); d_max = np.max(data);         # min and max of image
# cmin = (d_mean*d_min*d_med)**(1/3)     
        cmin = d_med - d_std; cmax = d_mean + d_std                  
        d_lower=d_min
        d_upper=(d_max*d_med)**0.5        

    
        #Making the Mask 
        mask = data.copy() #Setting mask initially to whole image
        image_centre=[int(mask.shape[1]/2),int(mask.shape[0]/2)] #Finding image centre by taking half of the length of each axis
        w=2000; h=2000; #Sub image size 2000x2000
        p=[(image_centre[0] - int(w/2)),(image_centre[1] - int(h/2))] #Placing the sub image in the middle of the image
        mask[:,:]= np.mean(data) 

        for u in range(p[0],p[0]+w): #For all pixels in the x direction between the LHS and RHS of the sub-image
            for v in range(p[1],p[1]+h): #For all pixels in the Y direction between the TOP and BOTTOM of the sub-image
                mask[u,v] = data[u,v]

        #Finding Sources Within the Mask
        sources = IRAFfind(mask)  #Only find sources within the mask
        for col in sources.colnames:  
            sources[col].info.format = '%.6g'  # for consistent table output
        sources.sort('flux') #Sorting the sources by flux, reverse
        sources.reverse()
        #Converting Sources within mask to RA-DEC
        pix_stars = np.transpose((sources['xcentroid'], sources['ycentroid'])) #List of all stars in pixel coords
        x_stars,y_stars = pix_stars[:,0],pix_stars[:,1] 
        ra, dec = wcs.wcs_pix2world(x_stars, y_stars, 1) #convert all stars to RA-DEC
        stars = np.transpose((ra,dec))
        
        RA.extend(ra)
        DEC.extend(dec)
        
coord_data = np.column_stack((RA, DEC))
Coords_df = pd.DataFrame(coord_data, columns = ['Right-Ascension' , 'Declination']) 
Coords_df = Coords_df.round({'Right-Ascension': 3, 'Declination': 3})
Coords_df = Coords_df.drop_duplicates(subset = ['Right-Ascension' , 'Declination'], keep = 'first')
Coords_df = Coords_df.sort_values(by = ['Right-Ascension', 'Declination'], ascending = False)
Coords_df.to_csv(os.path.join(local_dir, '1. ' + cluster_dir + "_Coord_data.csv"))

print("----------------------------------------------------------------------")
print("")
print("Finish")

#         #Plotting
#         # plot the solved image using the wcs projection
#                 plt.figure(figsize=(25,25))
#                 ax = plt.subplot(projection=wcs)
#                 mean_intensity = np.median(data)
#                 rng=1.025
#                 cmin = mean_intensity/rng; cmax = mean_intensity*rng;
#                 plt.imshow(data, vmin=cmin, vmax=cmax,cmap='gray')
#                 plt.grid(color='yellow', ls='solid')

#                 #Ploting apertures ands annuli
#                 fwhm = np.mean(sources['fwhm'])
#                 pix_radius = 4*fwhm/2 # 4 times the average star raduis to collect all the possible photons
#                 In_ann = pix_radius*1.2; Out_ann = (In_ann+1)*1.3 # annulus measurements from 1.2*aperture to 1.3*aperture
#                 apertures_labels = CircularAperture(pix_stars, r=pix_radius) # Setting aperture for flux to be the pixel radius of 4 time HWHM
#                 apertures_labels.plot(color='green', lw=2, alpha=0.5)
#                 ann_labels = CircularAnnulus(pix_stars, r_in=In_ann, r_out=Out_ann) #Setting Annulus for Background 
#                 ann_labels.plot(color='blue', lw=1, alpha=0.5)
#                 for p,l in enumerate(pix_stars[0:50]):
#                 plt.text(p[0]+10,p[1],sources['id'][l],fontsize=12,color='yellow') # plot id labels on the figure which match thos eof DAO starfinder


----------------------------------------------------------------------
----------------------------------------------------------------------
Opening file:  ngc-869_LIGHT_2021-03-12T19-54-09Z_I_120s_-5.0C_E_0016.fits
Calibrating...
This image is in the  I' filter
Calibration Completed! Starting Plate Solve...
Sending data to AstrometryNet server:
Solving........................
 -> Plate Solve Success! Solving took 25.5s
----------------------------------------------------------------------
----------------------------------------------------------------------
Opening file:  ngc-869_LIGHT_2021-02-02T20-10-56Z_G_120s_-4.9C_E_0004.fits
Calibrating...
This image is in the  G' filter
Calibration Completed! Starting Plate Solve...
Sending data to AstrometryNet server:
Solving...........................................................................................................................................................................................................................